## Mandate 4

**In mandate 3, I have used various supervised algorithms to predict the answer for given question.
In mandate 4 I have calculated the accuracy of these algorithms and have used root match technique to find the root of question and it's answer to check if it performs better than supervised algorithms.**

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')

from nltk import Tree
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Dataset for training the model¶

In [3]:
queans = pd.read_csv(r'/kaggle/input/trainq1/Book1.csv')

# Dataset for testing the model

In [153]:
testque=pd.read_csv(r'/kaggle/input/testq1/testque.csv')

In [154]:
testque

,Question
0,Which beaches are famous among tourists visiti...
1,What are some of the most popular beaches to v...
2,What is the significance of Rankala Lake in Ko...
3,What is Chikhaldara famous for?
4,How to reach the Elephanta Caves from Mumbai?
5,What is Nashik famous for?
6,What are some good accommodation options in Ga...
7,Is it possible to do camping in Ganpatipule?
8,What is the best time to visit Tadoba National...
9,Can I bring my own vehicle inside Tadoba Natio...


In [4]:
queans

,Question,Answer,target
0,What are some of the top tourist spots in Mumb...,Mumbai has several top popular tourist attract...,0
1,Which beaches are famous among tourists visiti...,"popular beaches in Maharashtra are Juhu Beach,...",1
2,Can you recommend some of the well-known beach...,"popular beaches in Maharashtra are Juhu Beach,...",1
3,Are there any highly-rated beaches in Maharash...,"popular beaches in Maharashtra are Juhu Beach,...",1
4,What are some of the most popular beaches to v...,"popular beaches in Maharashtra are Juhu Beach,...",1
...,...,...,...
513,What is the significance of the Ganesh Temple ...,The Ganesh Temple is dedicated to Lord Ganesha...,513
514,What is the Shivshrusti Museum?,The Shivshrusti Museum is a museum in Tuljapur...,514
515,What are the other places to visit near Tuljapur?,Some popular places to visit near Tuljapur are...,515
516,What is the history of Tuljapur?,Tuljapur has a rich history and has been menti...,516


****creating a list of all answers to create a voculabulary****

In [156]:
answer = queans['Answer'].tolist()

In [157]:
answer


['Mumbai has several top popular tourist attractions worth visiting, such as the Gateway of India, Marine Drive, Elephanta Caves, Juhu Beach, Chhatrapati Shivaji Terminus, Haji Ali Dargah and Siddhivinayak Temple.',
 'popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.',
 'popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.',
 'popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.',
 'popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.',
 'popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.',
 'Sinhgad fort near pune is considered popular among tourist',
 

In [158]:
import shutil

In [ ]:
# here we need to restructure working directory, so that script imports working properly
shutil.copytree("/kaggle/input/infersent/", "/kaggle/working/infersent")
! mv /kaggle/working/infersent/* /kaggle/working/

In [160]:
%%time

# TODO: add encoder to dataset as well
# If this cell freezes, probably you haven't enabled Internet access for the notebook
! mkdir encoder
! curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

mkdir: cannot create directory ‘encoder’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0   158M      0 --:--:-- --:--:-- --:--:--  158M
CPU times: user 39.1 ms, sys: 42.8 ms, total: 81.9 ms
Wall time: 3.24 s


## Load Model

In [161]:
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
W2V_PATH = '/kaggle/input/glove-840b-300d/glove.840B.300d.txt'
VOCAB_SIZE = 1e5  # Load embeddings of VOCAB_SIZE most frequent words
USE_CUDA = False  # Keep it on CPU if False, otherwise will put it on GPU

In [162]:
from models import InferSent
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

# ****Building a vocabulary of infersent model****

In [163]:
%%time
model = model.cuda() if USE_CUDA else model

model.set_w2v_path(W2V_PATH)

#model.build_vocab_k_words(K=VOCAB_SIZE)

CPU times: user 91 µs, sys: 0 ns, total: 91 µs
Wall time: 96.6 µs


In [164]:
model.build_vocab(answer, tokenize=True)

Found 1115(/1190) words with w2v vectors
Vocab size : 1115


In [165]:
embeddings = {}
for i in range(len(answer)):
    embeddings[answer[i]] = model.encode([answer[i]], tokenize=True)



In [166]:
question=queans["Question"].tolist()

In [167]:
question=queans["Question"].tolist()

****Creating a list of all Question to build the vocabulary****

In [168]:
model.build_vocab(question, tokenize=True)

Found 307(/321) words with w2v vectors
Vocab size : 307


In [169]:
for i in range(len(question)):
    embeddings[question[i]] = model.encode([question[i]], tokenize=True)

****Function to genearate embeddings of each question and answer****

In [170]:
def generate_embeddings(df):
    
    print("embedding the questions")
    df['quest_emb'] = df['Question'].apply(lambda x: embeddings[x] if x in embeddings else np.zeros(4096) )
    
    print("embedding the sentences")
    df['sent_emb'] = df['sentences'].apply(lambda x: [embeddings[item][0] if item in\
                                                           embeddings else np.zeros(4096) for item in x])
   
        
    return df 

In [171]:
embeddings.keys()

dict_keys(['Mumbai has several top popular tourist attractions worth visiting, such as the Gateway of India, Marine Drive, Elephanta Caves, Juhu Beach, Chhatrapati Shivaji Terminus, Haji Ali Dargah and Siddhivinayak Temple.', 'popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.', 'Sinhgad fort near pune is considered popular among tourist', 'The fort worth visiting in pune region is sinhgad fort.', 'The famous fort near Pune is Sinhagad Fort.', 'The distance between Mumbai and Mahabaleshware is approximately 260 km.', 'You can reach mahabaleshwar from mumbai by road. Alternatively, one can also take a train from Mumbai to Satara or Pune and then continue the journey by taxi or bus to Mahabaleshwar', 'Some of the highly-rated waterfalls in Maharashtra are Dudhsagar Falls, Thoseghar Falls, and Bhivpuri Falls.', 'Some of the well-known waterfalls in Maharashtra are Dudhsagar Falls, Thoseghar Falls, and

In [172]:
queans = queans.assign(sentences='NAN')
testque=testque.assign(sentences='NAN')
for i in range(len(queans)):
    queans['sentences'][i]=answer
    testque['sentences'][i]=answer

In [173]:
queans = generate_embeddings(queans)
testque=generate_embeddings(testque)

embedding the questions
embedding the sentences
embedding the questions
embedding the sentences


In [174]:
queans.head()

,Question,Answer,target,sentences,quest_emb,sent_emb
0,What are some of the top tourist spots in Mumb...,Mumbai has several top popular tourist attract...,0,[Mumbai has several top popular tourist attrac...,"[[0.10732491, 0.1332936, 0.16658722, 0.0084120...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
1,Which beaches are famous among tourists visiti...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.1396975, 0.03466962, 0.038167685, 0.010366...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
2,Can you recommend some of the well-known beach...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.071802296, 0.13915935, 0.03886863, 0.01120...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
3,Are there any highly-rated beaches in Maharash...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.09931967, 0.0888381, -0.0063716723, -0.004...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
4,What are some of the most popular beaches to v...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.13234368, 0.1332936, 0.14194623, 0.0032764...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."


****The dataframe below contains test questions****

In [175]:
testque



,Question,sentences,quest_emb,sent_emb
0,Which beaches are famous among tourists visiti...,[Mumbai has several top popular tourist attrac...,"[[0.1396975, 0.03466962, 0.038167685, 0.010366...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
1,What are some of the most popular beaches to v...,[Mumbai has several top popular tourist attrac...,"[[0.13234368, 0.1332936, 0.14194623, 0.0032764...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
2,What is the significance of Rankala Lake in Ko...,[Mumbai has several top popular tourist attrac...,"[[0.08341906, 0.12595488, -0.005576034, 0.0282...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
3,What is Chikhaldara famous for?,[Mumbai has several top popular tourist attrac...,"[[0.12592296, 0.054911118, -0.013959841, 0.032...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
4,How to reach the Elephanta Caves from Mumbai?,[Mumbai has several top popular tourist attrac...,"[[0.059352927, 0.062066663, 0.056393433, -0.01...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
5,What is Nashik famous for?,[Mumbai has several top popular tourist attrac...,"[[0.11737932, 0.008955083, 0.01506545, 0.03246...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
6,What are some good accommodation options in Ga...,[Mumbai has several top popular tourist attrac...,"[[0.079765186, 0.13247387, 0.071153, -0.038044...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
7,Is it possible to do camping in Ganpatipule?,[Mumbai has several top popular tourist attrac...,"[[0.08196658, 0.09579415, 0.010337181, 0.02262...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
8,What is the best time to visit Tadoba National...,[Mumbai has several top popular tourist attrac...,"[[0.116824865, 0.07716018, 0.045541942, 0.0239...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."
9,Can I bring my own vehicle inside Tadoba Natio...,[Mumbai has several top popular tourist attrac...,"[[0.09318808, 0.078734964, 0.04928308, -0.0127...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718..."


In [176]:
def similarity_cosine(x):
    cosine_distance = []
    for embeding in x["sent_emb"]:
        cosine_distance.append(spatial.distance.cosine(embeding,x["quest_emb"][0]))
    return cosine_distance   

In [177]:
def predictions(train):
    
    train["diffirence"] = (train["quest_emb"] - train["sent_emb"])**2
    train["cosine_sim"] = train.apply(similarity_cosine, axis = 1)
    train["euclidean_dis"] = train["diffirence"].apply(lambda x: list(np.sum(x, axis = 1)))
    del train["diffirence"]
    
    train["pred_euc"] = train["euclidean_dis"].apply(lambda y: np.argmin(y))  
    train["pred_cos"] = train["cosine_sim"].apply(lambda y: np.argmin(y))
     
    return train

In [178]:
predicted = predictions(queans)
testprediction=predictions(testque)

In [179]:
predicted.head()

,Question,Answer,target,sentences,quest_emb,sent_emb,cosine_sim,euclidean_dis,pred_euc,pred_cos
0,What are some of the top tourist spots in Mumb...,Mumbai has several top popular tourist attract...,0,[Mumbai has several top popular tourist attrac...,"[[0.10732491, 0.1332936, 0.16658722, 0.0084120...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.2687249183654785, 0.38381046056747437, 0.38...","[19.258896, 16.373264, 16.373264, 16.373264, 1...",7,63
1,Which beaches are famous among tourists visiti...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.1396975, 0.03466962, 0.038167685, 0.010366...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.2942333221435547, 0.2996787428855896, 0.299...","[20.176184, 13.645299, 13.645299, 13.645299, 1...",104,100
2,Can you recommend some of the well-known beach...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.071802296, 0.13915935, 0.03886863, 0.01120...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.2941240668296814, 0.3066006898880005, 0.306...","[20.368042, 13.471561, 13.471561, 13.471561, 1...",182,333
3,Are there any highly-rated beaches in Maharash...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.09931967, 0.0888381, -0.0063716723, -0.004...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.3073003888130188, 0.3211020827293396, 0.321...","[21.044308, 14.05007, 14.05007, 14.05007, 14.0...",7,342
4,What are some of the most popular beaches to v...,"popular beaches in Maharashtra are Juhu Beach,...",1,[Mumbai has several top popular tourist attrac...,"[[0.13234368, 0.1332936, 0.14194623, 0.0032764...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.267228901386261, 0.27867501974105835, 0.278...","[18.74886, 12.588791, 12.588791, 12.588791, 12...",173,333


In [180]:
testprediction.head(10)

,Question,sentences,quest_emb,sent_emb,cosine_sim,euclidean_dis,pred_euc,pred_cos
0,Which beaches are famous among tourists visiti...,[Mumbai has several top popular tourist attrac...,"[[0.1396975, 0.03466962, 0.038167685, 0.010366...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.2942333221435547, 0.2996787428855896, 0.299...","[20.176184, 13.645299, 13.645299, 13.645299, 1...",104,100
1,What are some of the most popular beaches to v...,[Mumbai has several top popular tourist attrac...,"[[0.13234368, 0.1332936, 0.14194623, 0.0032764...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.267228901386261, 0.27867501974105835, 0.278...","[18.74886, 12.588791, 12.588791, 12.588791, 12...",173,333
2,What is the significance of Rankala Lake in Ko...,[Mumbai has several top popular tourist attrac...,"[[0.08341906, 0.12595488, -0.005576034, 0.0282...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.39781653881073, 0.4421231150627136, 0.44212...","[25.691849, 18.021168, 18.021168, 18.021168, 1...",130,92
3,What is Chikhaldara famous for?,[Mumbai has several top popular tourist attrac...,"[[0.12592296, 0.054911118, -0.013959841, 0.032...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.4257095456123352, 0.45626044273376465, 0.45...","[26.942135, 18.971102, 18.971102, 18.971102, 1...",130,358
4,How to reach the Elephanta Caves from Mumbai?,[Mumbai has several top popular tourist attrac...,"[[0.059352927, 0.062066663, 0.056393433, -0.01...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.2874857187271118, 0.388012170791626, 0.3880...","[19.83777, 17.31601, 17.31601, 17.31601, 17.31...",13,57
5,What is Nashik famous for?,[Mumbai has several top popular tourist attrac...,"[[0.11737932, 0.008955083, 0.01506545, 0.03246...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.4946916699409485, 0.5290225744247437, 0.529...","[29.934765, 20.269821, 20.269821, 20.269821, 2...",130,73
6,What are some good accommodation options in Ga...,[Mumbai has several top popular tourist attrac...,"[[0.079765186, 0.13247387, 0.071153, -0.038044...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.3556176424026489, 0.3624919056892395, 0.362...","[23.495642, 15.766054, 15.766054, 15.766054, 1...",222,222
7,Is it possible to do camping in Ganpatipule?,[Mumbai has several top popular tourist attrac...,"[[0.08196658, 0.09579415, 0.010337181, 0.02262...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.4009968042373657, 0.3984304666519165, 0.398...","[25.80723, 17.188225, 17.188225, 17.188225, 17...",222,222
8,What is the best time to visit Tadoba National...,[Mumbai has several top popular tourist attrac...,"[[0.116824865, 0.07716018, 0.045541942, 0.0239...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.29473602771759033, 0.4005787968635559, 0.40...","[20.214306, 18.842087, 18.842087, 18.842087, 1...",279,279
9,Can I bring my own vehicle inside Tadoba Natio...,[Mumbai has several top popular tourist attrac...,"[[0.09318808, 0.078734964, 0.04928308, -0.0127...","[[0.18522227, 0.1957012, 0.1461941, 0.03426718...","[0.2879676818847656, 0.42926567792892456, 0.42...","[20.110151, 21.924507, 21.924507, 21.924507, 2...",290,290


In [181]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_sm')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [182]:
def update_dataframe(data):
    train = pd.DataFrame()
    
                  
    for k, sim in enumerate(data["cosine_sim"]):
        sim_list = ast.literal_eval(str(sim).replace("nan","1"))
        for i in range(len(sim_list)):
             train.loc[k, "column_cos"+"%s"%i] = sim_list[i]

         
    for k, dis in enumerate(data["euclidean_dis"]):
        distances = ast.literal_eval(str(dis))
        for i in range(len(distances)):
            train.loc[k, "column_euc"+"%s"%i] = distances[i]
   
  
            
    train["target"] = data["target"]
    return train

In [183]:
def update_dataframe_test(data):
    test = pd.DataFrame()
     
        
    for k, sim in enumerate(data["cosine_sim"]):
        sim_list = ast.literal_eval(str(sim).replace("nan","1"))
        for i in range(len(sim_list)):
             test.loc[k, "column_cos"+"%s"%i] = sim_list[i]

    for k, dis in enumerate(data["euclidean_dis"]):
        distances = ast.literal_eval(str(dis))
        for i in range(len(distances)):
            test.loc[k, "column_euc"+"%s"%i] = distances[i]
                    
   
    return test

****Converting the dataframe so that it will contain only cosine and euclidean distance and target answer index****

In [184]:
train = update_dataframe(queans)
test=update_dataframe_test(testque)

In [185]:
train

,column_cos0,column_cos1,column_cos2,column_cos3,column_cos4,column_cos5,column_cos6,column_cos7,column_cos8,column_cos9,...,column_euc364,column_euc365,column_euc366,column_euc367,column_euc368,column_euc369,column_euc370,column_euc371,column_euc372,target
0,0.268725,0.383810,0.383810,0.383810,0.383810,0.383810,0.333108,0.296749,0.428293,0.447088,...,16.403600,18.969260,12.951273,15.332931,14.966841,18.108532,11.494783,17.585125,15.367697,0
1,0.294233,0.299679,0.299679,0.299679,0.299679,0.299679,0.301721,0.362992,0.402899,0.535828,...,18.148808,20.877213,17.707910,19.854176,18.340763,19.806236,14.278935,22.020884,20.296500,1
2,0.294124,0.306601,0.306601,0.306601,0.306601,0.306601,0.371929,0.371414,0.430761,0.521847,...,15.231284,18.588957,15.723910,17.340260,15.474049,18.159870,13.092487,19.627148,17.356829,1
3,0.307300,0.321102,0.321102,0.321102,0.321102,0.321102,0.355673,0.342792,0.461045,0.516313,...,16.673035,19.394516,15.417397,17.863058,16.425568,19.751160,12.601875,18.695960,17.210543,1
4,0.267229,0.278675,0.278675,0.278675,0.278675,0.278675,0.327781,0.375039,0.421489,0.522287,...,15.696075,18.574245,16.390396,17.047000,15.288305,18.001648,12.435935,20.214058,17.331287,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,0.408582,0.464946,0.464946,0.464946,0.464946,0.464946,0.442080,0.461055,0.458309,0.593867,...,16.178312,18.401363,12.606665,13.189770,11.610755,18.000683,13.512022,19.219162,15.690332,368
369,0.379380,0.436716,0.436716,0.436716,0.436716,0.436716,0.411035,0.453418,0.414689,0.606662,...,15.585262,18.644848,13.592993,13.507340,14.262218,11.805460,13.876617,18.514334,15.282410,369
370,0.393282,0.406966,0.406966,0.406966,0.406966,0.406966,0.436595,0.418152,0.455959,0.533710,...,15.586356,19.094680,11.728466,11.696981,14.203885,17.044846,6.836786,16.584831,14.165956,370
371,0.320965,0.419850,0.419850,0.419850,0.419850,0.419850,0.434624,0.416161,0.435497,0.533734,...,14.736885,17.920204,13.821404,14.191124,13.139956,16.832043,14.618601,7.528361,13.324342,371


In [186]:
test

,column_cos0,column_cos1,column_cos2,column_cos3,column_cos4,column_cos5,column_cos6,column_cos7,column_cos8,column_cos9,...,column_euc363,column_euc364,column_euc365,column_euc366,column_euc367,column_euc368,column_euc369,column_euc370,column_euc371,column_euc372
0,0.294233,0.299679,0.299679,0.299679,0.299679,0.299679,0.301721,0.362992,0.402899,0.535828,...,23.797344,18.148808,20.877213,17.707910,19.854176,18.340763,19.806236,14.278935,22.020884,20.296500
1,0.267229,0.278675,0.278675,0.278675,0.278675,0.278675,0.327781,0.375039,0.421489,0.522287,...,19.344906,15.696075,18.574245,16.390396,17.047000,15.288305,18.001648,12.435935,20.214058,17.331287
2,0.397817,0.442123,0.442123,0.442123,0.442123,0.442123,0.440009,0.387282,0.384637,0.463524,...,23.901745,15.758508,22.744835,12.539141,14.383519,17.961752,20.715004,15.688486,20.351828,17.705204
3,0.425710,0.456260,0.456260,0.456260,0.456260,0.456260,0.429986,0.476701,0.403469,0.605716,...,22.746922,15.909698,21.418344,12.269343,11.990092,15.579623,17.063438,13.732658,18.713428,16.553265
4,0.287486,0.388012,0.388012,0.388012,0.388012,0.388012,0.432321,0.414145,0.373485,0.441551,...,23.133970,16.228550,23.124783,13.107016,17.244347,17.161140,21.731432,17.092403,19.657120,19.580425
5,0.494692,0.529023,0.529023,0.529023,0.529023,0.529023,0.459471,0.444398,0.400029,0.601094,...,29.326628,20.672121,27.072876,15.335614,17.291382,20.820364,21.819110,17.107052,23.961094,22.067430
6,0.355618,0.362492,0.362492,0.362492,0.362492,0.362492,0.407046,0.417424,0.478934,0.564188,...,22.745540,19.592133,20.994125,16.795435,18.325098,17.396145,18.737840,10.818028,20.591190,18.741009
7,0.400997,0.398430,0.398430,0.398430,0.398430,0.398430,0.446235,0.460339,0.486596,0.592144,...,21.781948,19.391270,19.234436,17.212990,19.242012,16.046673,21.537111,15.729493,20.874996,17.666204
8,0.294736,0.400579,0.400579,0.400579,0.400579,0.400579,0.395341,0.380038,0.364788,0.523741,...,18.694588,15.349836,16.947540,15.870299,17.359905,12.647583,19.948854,16.420582,17.338330,15.724229
9,0.287968,0.429266,0.429266,0.429266,0.429266,0.429266,0.422474,0.426919,0.433195,0.539913,...,19.325253,15.821306,14.836863,18.521221,19.104973,11.327307,19.353569,18.024420,16.204054,14.535002


****Replacing NaN of Cosine similarity with 1 and Euclidean distance with 60 :****

In [189]:
set1 = train.iloc[:,:518].fillna(60)
set2 = train.iloc[:,518:].fillna(1)
set3=test.iloc[:,:518].fillna(60)
set4=test.iloc[:,518:].fillna(1)

****Merging the dataframe after replacing NaN values****

In [190]:
train2 = pd.concat([set1, set2], axis=1)
test2=pd.concat([set3, set4], axis=1)
train2 = train2.reindex(set1.index)
test2 = test2.reindex(set3.index)

# Applying various supervised algorithm to generate answer for a given question

In [192]:
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train2.iloc[:,:-1])
test_x= scaler.fit_transform(test2)
train_y=train.iloc[:,-1]

In [193]:
# train_x, test_x, train_y, test_y = train_test_split(X,
# train.iloc[:,-1], train_size=0.8, random_state = 5)

****This is the expected answer Id for test questions****

In [217]:
test_y=[1,1,273,358,64,73,221,222,279,293,370]

In [196]:
lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg',  max_iter = 15000)
lr.fit(train_x, train_y)
pred=lr.predict(train_x[2].reshape(1,746))#test_x
#print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, pred))

In [211]:
pred=lr.predict(train_x)#test_x
print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, pred))

Multinomial Logistic regression Train Accuracy :  0.967828418230563


In [ ]:
pred=lr.predict(test_x)

In [218]:
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, pred))

Multinomial Logistic regression Test Accuracy :  0.8181818181818182


In [219]:
pred

array([  1, 339, 273, 358,  64, 125, 221, 222, 279, 293, 370])

# Answer generated by Logistic Regression

In [225]:
 for i in range(len(test_x)):
        print("Question : " + testque["Question"][i])
        print("Answer by Logistic Regression : " + answer[pred[i]])
        print("\n")

Question : Which beaches are famous among tourists visiting Maharashtra?
Answer by Logistic Regression : popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.


Question : What are some of the most popular beaches to visit in Maharashtra?
Answer by Logistic Regression : Chiplun is home to several stunning waterfalls that are a must-visit for nature lovers. The popular waterfalls to visit in Chiplun include Sawatsada Waterfalls, Parshuram Waterfalls, and Unhavare Hot Water Springs.


Question : What is the significance of Rankala Lake in Kolhapur?
Answer by Logistic Regression : Rankala Lake is a man-made lake in Kolhapur and is a popular spot for picnics and boating. It is also surrounded by several temples and parks.


Question : What is Chikhaldara famous for?
Answer by Logistic Regression : Chikhaldara is famous for its scenic beauty, wildlife sanctuary, lakes, and waterfalls.


Question : How to r

In [229]:
rf = RandomForestClassifier(min_samples_leaf=9, n_estimators=90)
rf.fit(train_x, train_y)

ypredRf=rf.predict(test_x)
#print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, ypredRf))

In [210]:
ypredRf = rf.predict(train_x)
print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, ypredRf))

Random Forest Train Accuracy :  0.9329758713136729


In [231]:
ypredRf=rf.predict(test_x)
ypredRf

array([  1,   1, 247, 161, 253, 125,   1, 318,  91, 293, 370])

In [230]:
print("Random Forest Test Accuracy : ", metrics.accuracy_score(test_y, ypredRf))

Random Forest Test Accuracy :  0.36363636363636365


# Answer generated by Random Forest

In [226]:
 for i in range(len(test_x)):
        print("Question : " + testque["Question"][i])
        print("Answer by Random Forest : " + answer[ypredRf[i]])
        print("\n")

Question : Which beaches are famous among tourists visiting Maharashtra?
Answer by Random Forest : popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.


Question : What are some of the most popular beaches to visit in Maharashtra?
Answer by Random Forest : popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.


Question : What is the significance of Rankala Lake in Kolhapur?
Answer by Random Forest : The height of Kalsubai Peak in Igatpuri is 1,646 meters (5,400 feet) above sea level.


Question : What is Chikhaldara famous for?
Answer by Random Forest : Rajmachi can be reached by trekking or hiring a private vehicle from Lonavala or Karjat.


Question : How to reach the Elephanta Caves from Mumbai?
Answer by Random Forest : The distance between Mumbai and Konkan varies depending on the destination, but it is usual

In [203]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [204]:
from sklearn.metrics import accuracy_score
estimator = []
# estimator.append(('LR', 
#                   LogisticRegression(solver ='lbfgs', 
#                                      multi_class ='multinomial', 
#                                      max_iter = 10000)))
estimator.append(('SVC', SVC(gamma ='auto', probability = True)))
estimator.append(('LR', LogisticRegression(multi_class='multinomial', solver='newton-cg',  max_iter = 15000)))
estimator.append(('RFC', RandomForestClassifier(min_samples_leaf=9, n_estimators=90)))
  
# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='soft')
vot_hard.fit(train_x, train_y)
y_pred = vot_hard.predict(test_x)
#print("Voting Classifer Train Accuracy : ", metrics.accuracy_score(train_y, y_pred))
print(y_pred)

[  1 339 273 358  64 125 221 222 279 293 370]


In [207]:
y_pred = vot_hard.predict(train_x)
print("Voting Classifer Train Accuracy : ", metrics.accuracy_score(train_y, y_pred))

Voting Classifer Train Accuracy :  0.9624664879356568


In [222]:
y_pred = vot_hard.predict(test_x)
print("Voting Classifer Train Accuracy : ", metrics.accuracy_score(test_y, y_pred))

Voting Classifer Train Accuracy :  0.8181818181818182


# Answer generated by Voting Classifier

In [205]:
 for i in range(len(test_x)):
        print("Question : " + testque["Question"][i])
        print("Answer by voting classifier : " + answer[y_pred[i]])
        print("\n")

Question : Which beaches are famous among tourists visiting Maharashtra?
Answer by voting classifier : popular beaches in Maharashtra are Juhu Beach, Alibaug Beach, and Kashid Beach, Ganapatipule Beach, Tarkarli Beach, and Dahanu-Bordi Beach.


Question : What are some of the most popular beaches to visit in Maharashtra?
Answer by voting classifier : Chiplun is home to several stunning waterfalls that are a must-visit for nature lovers. The popular waterfalls to visit in Chiplun include Sawatsada Waterfalls, Parshuram Waterfalls, and Unhavare Hot Water Springs.


Question : What is the significance of Rankala Lake in Kolhapur?
Answer by voting classifier : Rankala Lake is a man-made lake in Kolhapur and is a popular spot for picnics and boating. It is also surrounded by several temples and parks.


Question : What is Chikhaldara famous for?
Answer by voting classifier : Chikhaldara is famous for its scenic beauty, wildlife sanctuary, lakes, and waterfalls.


Question : How to reach the

****Constructing the root mapping of question and answer****

In [4]:
def sent_root_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [sent_root_tree(child) for child in node.children])
    else:
        return node.orth_

**Generated root map tree for first question**

In [5]:
[sent_root_tree(sent.root).pretty_print()  for sent in en_nlp(queans['Question'][0]).sents]

             are                                 
  ____________|_____                              
 |    |            some                          
 |    |             |                             
 |    |             of                           
 |    |             |                             
 |    |           spots                          
 |    |    _________|_________________            
 |    |   |   |     |      |         are         
 |    |   |   |     |      |      ____|_____      
 |    |   |   |     |      in    |        worth  
 |    |   |   |     |      |     |          |     
What  ?  the top tourist Mumbai that     visiting



[None]

In [6]:
[sent_root_tree(sent.root).pretty_print()  for sent in en_nlp(queans['Answer'][0]).sents]

             has                                                                                                                                                           
   ___________|_____________________                                                                                                                                        
  |     |                      attractions                                                                                                                                 
  |     |      _____________________|________________________________________                                                                                               
  |     |     |     |     |         |       |     |                          as                                                                                            
  |     |     |     |     |         |       |     |       ___________________|_________________________                                   

[None]

**Printing roots and sub roots**

In [96]:

    roots = [st.stem(chunk.root.head.text.lower()) for chunk in en_nlp(queans['Answer'][10]).sents]#sent.noun_chunks]
    print(roots)

['reach', 'tak']


In [213]:
question_root = st.stem(str([sent.root.head.text.lower() for sent in en_nlp(queans['Question'][10]).noun_chunks]))
print(question_root)

['reach', 'reach', 'from']


**Root match technique generated same root for the above question and answer which is 'reach', but for many question answer pair it is enable to find common root hence it is  not as effective as supervised algorithms**